# Проектная работа:
# "Рынок заведений общественного питания Москвы"

## Описание проекта

Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.

### Задачи проекта:

- Подготовить исследование рынка Москвы, 
- найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.

Доступен датасет с заведениями общественного питания Москвы, составленный на основе данных сервисов Яндекс Карты и Яндекс Бизнес на лето 2022 года. Информация, размещённая в сервисе Яндекс Бизнес, могла быть добавлена пользователями или найдена в общедоступных источниках.

### План проекта:
 1. Загрузить данные и изучите общую информацию
 2. Выполнить предобработку данных
 3. Анализ данных
 4. Детализация исследования: открытие кофейни
 5. Подготовка презентации

In [ ]:
#модули
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import folium
from folium import Map, Choropleth, Marker
from folium.plugins import MarkerCluster
import warnings
import json
warnings.filterwarnings('ignore')

### 1. Загрузить данные и изучите общую информацию

##### Задачи:
- Загрузить данные о заведениях общественного питания Москвы. (Путь к файлу: /datasets/moscow_places.csv)
- Изучить общую информацию о датасете. Сколько заведений представлено? Что можно сказать о каждом столбце? Значения какого типа они хранят? 

##### Цель:
- Прочитать датасет и ознакомиться с ним. 

In [ ]:
try:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/moscow_places.csv')  
except:
    pass

In [ ]:
len(df.index)

В датафрейме представлено 8406 строк

In [ ]:
df.info()

In [ ]:
df.head(5)

В датафрейме 14 столбцов, в которых:

- ***name*** - название заведения, тип данных - object, пропусков - нет, ***столбец в порядке.***
- ***category*** - категория заведения (например «кафе», «пиццерия» или «кофейня»), тип данных - object, пропусков - нет, ***столбец в порядке.***
- ***address*** - адрес заведения, тип данных - object, пропусков - нет, ***столбец в порядке.***
- ***district*** - административный район, в котором находится заведение тип данных - object, пропусков - нет, ***столбец в порядке.***
- ***hours*** - нформация о днях и часах работы, тип данных - object, пропусков ~ 6%, ***столбец необходимо привести в порядок, допустимое количество пропусков.***
- ***lat*** - широта географической точки, тип данных - float64, в которой находится заведение, пропусков - нет, ***столбец в порядке.***
- ***lng*** - долгота географической точки, тип данных - float64, в которой находится заведение, пропусков - нет, ***столбец в порядке.***
- ***rating*** - рейтинг заведения по оценкам пользователей в Яндекс Картах, тип данных - float64, пропусков - нет, ***столбец в порядке.***
- ***price*** - категория цен в заведении,тип - object, пропусков ~ 60%, ***столбец необходимо привести в порядок.***
- ***avg_bill*** - строка, которая хранит среднюю стоимость заказа в виде диапазона, тип данных - object, пропусков ~ 60%, ***столбец необходимо привести в порядок.***
- ***middle_avg_bill*** - число с оценкой среднего чека, тип данных - float64, пропусков ~ 60%, ***столбец необходимо привести в порядок.***
- ***middle_coffee_cup*** - число с оценкой одной чашки капучино,  тип данных - float64, пропусков ~ 90%, ***столбец необходимо привести в порядок.***
- ***chain*** - число, которое показывает, является ли заведение сетевым, тип данных - float64, пропусков нет, ***столбец необходимо привести в порядок.***
- ***seats*** - количество посадочных мест, тип данных - float64, пропусков ~ 45%, ***столбец необходимо привести в порядок.***

### Вывод

В датафрейме представлено 8406 строк в 14 столбцах, которые необходимо проработать.

Необходимо проделать следующую работу с данными в столбцах и строках:
- ***hours*** - перевести в удобный формат работы со временем.
- ***price*** -  заполнить пропуски на основании avg_bill, перевести в int - значения по четырехбальной шкале.
- ***avg_bill*** - заполнить пропуски значениями по типу заведения.
- ***middle_avg_bill*** - заполнить пропуски значениями по типу заведения.
- ***middle_coffee_cup*** - заполнить пропуски значениями по типу заведения.
- ***chain*** - заменить 0 и 1 на True и False.
- ***seats*** - заполнить пропуски значениями по типу заведения.

### 2. Выполнить предобработку данных

##### Задачи:
- Отработать пропуски, внести необходимые изменения в данные.
- Изучить дубликаты в данных. 
- Выполнить предобработку данных:

    - Создать столбец street с названиями улиц из столбца с адресом.
    - Создать столбец is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно (24/7):
        - логическое значение True — если заведение работает ежедневно и круглосуточно;
        - логическое значение False — в противоположном случае.

##### Цель:
- Подготовить датасет для дальнейшей работы.

##### 2.1 Обработка пропусков, внесение изменений в тип данных.

In [ ]:
df['price'].unique()

In [ ]:
#замена значенй в столбце price
df['price'] = df['price'].replace('низкие', 1)
df['price'] = df['price'].replace('средние', 2)
df['price'] = df['price'].replace('выше среднего', 3)
df['price'] = df['price'].replace('высокие', 4)

In [ ]:
#обработка пропусков в столбце price по типу заведения
#может ввести дальнейший анализ в заблуждение, так как этот столбец зависит от avg_bill.

In [ ]:
#обработка пропусков в столбце avg_bill по типу заведения на данном этапе
#методом заполнения медианным значением для заведения может ввести в заблуждение дальнейший анализ
#оставим пропуски как есть

In [ ]:
#обработка пропусков в столбце middle_coffee_cup по типу заведения
#может ввести дальнейший анализ в заблуждение, так как этот столбец зависит от avg_bill.

In [ ]:
#обработка пропусков в столбце middle_avg_bill по типу заведения
#может ввести дальнейший анализ в заблуждение, так как этот столбец зависит от avg_bill.

In [ ]:
#замена значенй в столбце chain
df['chain'] = df['chain'].replace([0, 1], [False, True])
#df['chain'] = df['chain'].replace(0, False)
#df['chain'] = df['chain'].replace(1, True)

In [ ]:
#обработка пропусков в столбце seats проходит следующим образом: вероятнее всего,
#в таких заведениях отсутствуют посадочные места
#df['seats'] = df['seats'].fillna(0)
#df['seats'] = df['seats'].astype(int)
#вернемся к данному столбцу позже

In [ ]:
columns = df.columns
for i in columns:
    print(f'Количество пропусков в столбце {i} - {df[i].isna().sum()}, что составляет - {df[i].isna().sum()/len(df) * 100:.2f}%')

Цены в дальнейшем, при необходимости, можно предугадать в зависимости от места расположения заведения.

##### 2.2 Дубликаты датафрейма

In [ ]:
print('Количество дубликатов: ' + str(df.duplicated().sum()))

##### 2.3 Создание столбца street с названиями улиц из столбца с адресом

In [ ]:
df['street'] = df['address'].apply(lambda x: x.split(',')[1].strip())

##### 2.4 Создание столбца is_24/7 с обозначением ежедневно работающих круглосуточных и некруглосуточных заведений

In [ ]:
df['is_24/7'] = df['hours'].apply(lambda x: True if x == 'ежедневно, круглосуточно' else False)

### Вывод

В датафрейме дубликатов нет. Созданы дополнительные столбцы по техническому заданию.

### 3. Анализ данных

##### Задачи:

- Выяснить, какие категории заведений представлены в данных. Исследовать количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее. Построить визуализации. Рассмотреть распределение заведений по категориям.
- Исследовать количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее. Построить визуализации. Проанализировать результаты и сделать выводы.
- Рассмотреть и изобразить соотношение сетевых и несетевых заведений в датасете.
- Выяснить, какие категории заведений чаще являются сетевыми. Исследовать данные и построить график.
- Сгруппировать данные по названиям заведений и найти топ-15 популярных сетей в Москве. Построить подходящую для такой информации визуализацию.
- Узнать какие административные районы Москвы присутствуют в датасете. Отобразить общее количество заведений и количество заведений каждой категории по районам. Проиллюстрировать эту информацию одним графиком.
- Визуализировать распределение средних рейтингов по категориям заведений. Сделать вывод, сильно ли различаются усреднённые рейтинги в разных типах общепита?
- Построить фоновую картограмму (хороплет) со средним рейтингом заведений каждого района. Границы районов Москвы, которые встречаются в датасете, хранятся в файле admin_level_geomap.geojson.
- Отобразить все заведения датасета на карте с помощью кластеров средствами библиотеки folium.
- Найти топ-15 улиц по количеству заведений. Построить график распределения количества заведений и их категорий по этим улицам. Проиллюстрировать эту информацию одним графиком.
- Найти улицы, на которых находится только один объект общепита. Что можно сказать об этих заведениях?
- Значения средних чеков заведений хранятся в столбце middle_avg_bill. Эти числа показывают примерную стоимость заказа в рублях, которая чаще всего выражена диапазоном. Посчитать медиану этого столбца для каждого района. Использовать это значение в качестве ценового индикатора района. Построить фоновую картограмму (хороплет) с полученными значениями для каждого района. Проанализируйте цены в центральном административном округе и других. Как удалённость от центра влияет на цены в заведениях?
- Собрать наблюдения по вопросам выше в один общий вывод.

##### Цель:

- Получить необходимую информацию из датасета, провести анализ по полученным данным.

Подготовим функцию для визуализации необходимой информации

In [ ]:
def display_countplot(df, column, title, ylabel, xlabel):     
    plt.figure(figsize=(18, 8))
    
    palette = sns.color_palette('Greens', df[column].nunique()) 
    palette.reverse() 
    #столбцы в порядке убывания
    ax = sns.countplot(x=column, data=df, order = df[column].value_counts().index, palette=palette)
    # разворот названий столбцов, размер шрифта по оси х
    plt.xticks(rotation=90, fontsize=16)
    #шрифт по оси y
    plt.yticks(fontsize=16)
    # подписи ко всем осям и самому графику
    ax.set_title(title, fontsize=18)
    ax.set_ylabel(ylabel, fontsize=18)
    ax.set_xlabel(xlabel, fontsize=18)

##### 3.1 Визуализация и анализ по категории заведения

In [ ]:
display_countplot(df, 'category',\
                  'Количество заведений по категориям',\
                  'Количество заведений в категории',\
                  'Название категории')

Кафе(2378шт), рестораны(2043шт) и кофейни(1413шт) являются достаточно популярными категориями заведений на фоне других типов.
Отсальных типов заведений - много меньше тысячи.

##### 3.2 Визуализация и анализ по посадочным местам в заведении

In [ ]:
plt.figure(figsize=(18, 10))
palette = sns.color_palette('Greens', df['category'].nunique())
palette.reverse()
box_order = df.groupby('category')['seats'].median().sort_values(ascending=False).index
ax = sns.boxplot(x='category', y='seats', data=df, order=box_order, palette=palette, showfliers=False) 
ax.set_ylabel('Количество мест', fontsize=18)
ax.set_xlabel('Тип заведения', fontsize=18)
plt.xticks(fontsize=14)
ax.set_title('Количество посадочных мест по категориям', fontsize=18);

In [ ]:
df.groupby('category')['seats'].median().sort_values(ascending=False)

Построив визуализацию и упорядочив категории по медианным значениям видно, что в ресторанах, барах и кофейнях медианные значения количества посадочных мест более 80. Наименьшее медианное значение по посадочным местам наблюдается в кафе, пиццериях и булочных - в диапазоне 50-60 мест.

##### 3.3 Визуализация и анализ по принадлежности к сетевой принадлежности

In [ ]:
chain_rat = df.groupby('chain', as_index=False)['chain'].count()
chain_rat = chain_rat.reset_index()
chain_rat.columns = ['Тип заведения', 'Количество']
chain_rat['Тип заведения'] = chain_rat['Тип заведения'].apply(lambda x: 'несетевое' if x == 0 else 'сетевое')
chain_rat

In [ ]:
plt.figure(figsize=(5, 5))
y = chain_rat['Количество']
lables = ['Сеть', 'Не сеть']
colors = ['DarkGreen', 'LightGreen']
plt.pie(y, labels = lables, colors=colors, autopct='%1.0f%%', textprops={'fontsize': 30})
plt.show() 

Сетевых заведений на порядок выше чем несетевых.

In [ ]:
df_chain = df[['category','chain']]
df_chain['chain'] = df_chain['chain'].apply(lambda x: 'несетевое' if x == 0 else 'сетевое')
df_g = df_chain.groupby(['category', 'chain']).size().reset_index()
df_g['percentage'] = df_chain.groupby(['category', 'chain']).size().groupby(level=0).apply(
    lambda x: 100 * x / float(x.sum())).values
df_g.columns = ['category', 'chain', 'Counts', 'Percentage']
fig = px.bar(
    df_g, x='category', 
    y=['Counts'], color='chain', 
    text=df_g['Percentage'].apply(
        lambda x: '{0:1.2f}%'.format(x)),
       color_discrete_sequence=px.colors.sequential.Darkmint,
                       labels={
                     "value": "Количество заведений",
                     "category": "Название категории",
                     "chain": "сетевое / несетевое"
                 },
                title="Отношение сетевых к несетевым заведениям по категориям")
fig.update_traces(textfont_size=20, textangle=0)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

Кафе, рестораны, бары и столовые чаще других категорий являются несетевыми заведениями – половина и более завезений этой категории не принадлежит к сетям.

##### 3.4 Визуализация и анализ топ-15 популярных сетей в Москве

In [ ]:
top_15 = df.query('chain == 1').groupby(['name', 'category'])['name'].count().sort_values(ascending=False).to_frame().head(15)
top_15.columns = ['count']
top_15 = top_15.reset_index(level=0).reset_index(level=0).sort_values(by='category')
top_15

In [ ]:
top_15_category = top_15.groupby('category')['count'].sum().to_frame().reset_index(level=0)
top_15_category

In [ ]:
top_15 = df.query('chain == 1').pivot_table(index=['category', 'name'], values='chain', aggfunc='count')\
.reset_index().sort_values(by='chain', ascending=False).head(15)

fig = px.sunburst(top_15, path=['category', 'name'], values='chain',\
                  
                  color_discrete_sequence=px.colors.qualitative.Pastel2
                  )
fig.update_layout(
    margin = dict(t=10, l=10, r=10, b=10),\
    title='Топ 15 заведений по категориям.',\
    title_xanchor="auto",\
    title_y=0
)
fig.show()

Большая часть заведений относится к кофейням: среди топ-15 популярных сетей 45% (350) заведений принадлежат к категории кофейня. Наименее популярная категория 3% (25) – булочная. В целом относительное распределение заведений по категориям имеет следующий вид:
кофейня – 45% (350)
ресторан – 20% (157)
пиццерия – 19,5% (150)
кафе – 11,5% (89)
булочная – 3.2% (25)

##### 3.5 Визуализация и анализ районов в Москве

In [ ]:
df['district'].unique()

In [ ]:
district_abbr = {
    'Северный административный округ': 'САО',
    'Северо-Западный административный округ': 'СЗАО',
    'Западный административный округ': 'ЗАО',
    'Восточный административный округ': 'ВАО',
    'Юго-Западный административный округ': 'ЮЗАО',
    'Южный административный округ': 'ЮАО',
    'Юго-Восточный административный округ': 'ЮВАО',
    'Центральный административный округ': 'ЦАО',
    'Северо-Восточный административный округ': 'СВАО',
    'Зеленоградский административный округ': 'ЗАО'
}

df['district_cont'] = df['district'].replace(district_abbr)

In [ ]:
regions_category = df.groupby(['district_cont', 'category'], as_index=False)['name'].count()\
.sort_values(by= ['district_cont', 'name'], ascending=False)
regions_category

In [ ]:
data = df[['district_cont','category']]
df_g = data.groupby(['district_cont', 'category']).size().reset_index()
df_g['percentage'] = data.groupby(['district_cont','category']).size().groupby(level=0).apply(lambda x: 100 * x / float(x.sum())).values
df_g.columns = ['district_cont', 'category', 'counts', 'percentage']
fig = px.bar(df_g, x='district_cont', y=['counts'], color='category', text=df_g['percentage'].apply(lambda x: '{0:1.2f}%'.format(x)),\
       color_discrete_sequence=px.colors.sequential.Darkmint,\
                       labels={
                     'value': 'Количество заведений',
                     'district_cont': 'Pайон',
                     'category': 'Категория'
                 },
                title='Процент категорий в разных округах',
      height= 1000, 
            )
fig.update_traces(textfont_size=50, textangle=0)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig

На фоне всех остальных административных округов заметно выделяется центральный административный: в нем не только больше заведений в сравнении с остальными округами, но и соотношение по категориям заведений тоже имеет различие в определенных категориях. В центральном административном округе значительно больше баров (в 2-3 раза) и ресторанов (примерно на 10% в сравнении с остальными округами). Но при этом популярность кафе в центральном административном округе заметно ниже, чем в остальных округах. Больше всего кафе в юго-восточном административном округе

##### 3.6 Визуализация и анализ распределения рейтингов заведения

In [ ]:
avg_rating = df.groupby('category', as_index=False)['rating'].mean().sort_values(by='rating', ascending=False)
avg_rating.columns = ['category', 'avg_rating']
avg_rating

In [ ]:
def display_barplot(df, column_1, column_2, title, ylabel, xlabel, ylim_bott, ylim_top):
    plt.figure(figsize=(18, 8))
    palette = sns.color_palette('Greens', df[column_1].nunique())
    palette.reverse()
    ax = sns.barplot(x=df[column_1], y=df[column_2], palette=palette)
    ax.set_xlabel(xlabel, fontsize=18)
    ax.set_ylabel(ylabel, fontsize=18)
    ax.set_title(title, fontsize=18);
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    ax.set_ylim(ylim_bott, ylim_top)
    
    for i in ax.patches:
        _x = i.get_x() + i.get_width() / 2
        _y = i.get_y() + i.get_height() + (i.get_height()*0.01)
        value = '{:.2f}'.format(i.get_height())
        ax.text(_x, _y, value, ha="center")

In [ ]:
display_barplot(avg_rating, 'category', 'avg_rating',\
               'Средний рейтинг по категориям заведений',\
               'Средний рейтинг',\
               'Категория заведения',\
               4, 4.5)

Средние рейтинги по категориям отличаются: наивысшие оценки получают бары и пабы, а самые низкие оценки ставят посетители заведений быстрого питания.

##### 3.7 Построение картограммы со средним рейтингом заведений каждого района.

In [ ]:
mid_rating_df = df.pivot_table(index='district', values='rating', aggfunc='mean').reset_index()
mid_rating_df = mid_rating_df.sort_values(by='rating', ascending=False)
mid_rating_df

In [ ]:
#работал локально, с другого источника не вышло подключить json
#r'C:\Users\soibr\OneDrive\Рабочий стол\jupyter notebook\prj_data_history\admin_level_geomap.geojson'
with open('/datasets/admin_level_geomap.geojson', encoding='utf-8') as a:
    geo_json = json.load(a)
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

def f_Choropleth(data,columns,name):
    Choropleth(
        geo_data=geo_json,
        data=data,
        columns=columns,
        key_on='feature.name',
        fill_color='YlGn',
        fill_opacity=0.8,
        legend_name=name,
    ).add_to(m)
    return m

In [ ]:
f_Choropleth(mid_rating_df,['district', 'rating'],'Медианный рейтинг заведений по районам')

##### 3.7.1 Вывод всех заведений на карте

In [ ]:
marker_cluster = MarkerCluster().add_to(m)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

df.apply(create_clusters, axis=1)
m

##### Вывод

По данной визуализации можно детальнее рассмотреть нахождение заведений в районах административных округов.

##### 3.8 Визуализация и анализ топ-15  улиц в Москве

In [ ]:
street_df = df.groupby(['street'], as_index=False)['name'].count().sort_values(by='name', ascending=False).head(15)
street_df = street_df.sort_values(by='name', ascending=False)
street_df.columns = ['street', 'counts']
street_df

In [ ]:
category_df = df.groupby(['street', 'category'], as_index=False)['name'].count().sort_values(by='name', ascending=False)
category_df = category_df.sort_values(by='name', ascending=False)
category_df.columns = ['street', 'category', 'counts']
category_df.sort_values(by='counts', ascending=False, inplace=True)
category_df

In [ ]:
df_merge = category_df.merge(street_df, how='inner', on='street')
df_merge

In [ ]:
df_merge['percent'] = df_merge['counts_x'] / df_merge['counts_y'] * 100
df_merge['percent'] = df_merge['percent'].apply(lambda x: '{0:1.2f}%'.format(x))  
df_merge.columns = ['street', 'category', 'count', 'count_street', 'percent']
df_merge

In [ ]:
fig = px.bar(df_merge, x='street', y=['count'], color='category', text=df_merge['percent'],\
                       labels={
                     "value": "Количество заведений",
                     "street": "Название улицы",
                     "category": "Категория"
                 },
    title="Отношение категорий в разных административных округах",\
    color_discrete_sequence=px.colors.sequential.Darkmint,\
    height= 1000)
fig.update_traces(textfont_size=20, textangle=0)
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()

Открыв карту москвы, можно выяснить, что длина улицы влияет на количество заведений. При наличии данных о длинне улиц можно было бы визуализировать зависимость. 

##### 3.9 Улицы с одним объектом общепита

In [ ]:
alone_obj = df.pivot_table(index='street', values='name', aggfunc='count').reset_index()
alone_obj.query('name==1')

Всего 458 улиц с одним объектом общепита.

In [ ]:
alone_obj = df.pivot_table(index=['district_cont', 'street'], values='name', aggfunc='count').reset_index()
alone_obj = alone_obj.query('name==1')
alone_obj

In [ ]:
display_countplot(alone_obj, 'district_cont',\
    'Количество улиц с одним заведением по округам',\
    'Название округа',\
    'Количество улиц с одним заведением')

Округ с большим количеством улиц с одним заведением - ЦАО.

##### 3.10 Медиана средних чеков для каждого района

In [ ]:
avg_bill = df.pivot_table(index='district', values='middle_avg_bill', aggfunc='median').reset_index()
avg_bill

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)
f_Choropleth(avg_bill,['district', 'middle_avg_bill'],'Медианный средний чек по районам')

### Вывод

### 4. Детализация исследования: открытие кофейни

##### Задачи:
Выяснить
- Сколько всего кофеен в датасете. В каких районах их больше всего, каковы особенности их расположения.
- Есть ли круглосуточные кофейни.
- Какие у кофеен рейтинги? Как они распределяются по районам.
- На какую стоимость чашки капучино стоит ориентироваться при открытии и почему.
- Построить визуализации.Дать рекомендацию для открытия нового заведения.

##### Цель:
- Определить, ожидает ли основателей фонда «Shut Up and Take My Money» успех.

##### 4.1 Сколько всего кофеен в датасете. В каких районах их больше всего, каковы особенности их расположения.

In [ ]:
coffee_count = df.query('category == "кофейня"').pivot_table(index='district', values='name', aggfunc='count').reset_index().sort_values(by='name', ascending=False)
coffee_count.columns = ['district', 'count']
coffee_count

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
f_Choropleth(coffee_count,['district', 'count'],'плотность распределения кофеен по районам')

In [ ]:
coffee_count['count'].sum()

Всего таких заведений в датасете - 1413

##### 4.2 Есть ли круглосуточные кофейни.

In [ ]:
round(df.query('category == "кофейня"')['is_24/7'].mean()*100, 2)

Заведений с таким графиком - 4.18% от общего числа заведений.

In [ ]:
df.query('category == "кофейня"')['hours'].value_counts().head(10)

In [ ]:
cofee_24 = df.query('category == "кофейня" & hours == "ежедневно, круглосуточно"')['hours'].value_counts()#.to_frame().reset_index(level=0)
cofee_24

##### Вывод

Круглосуточные коофейни – редкое явление. Таких - 4,2%, а именно – 59 заведений. В основном график таких заведений такого типа - с утра и до позднего вечера.

##### 4.3 Какие у кофеен рейтинги? Как они распределяются по районам.

In [ ]:
coffee_rating = df.query('category == "кофейня"').pivot_table(index='district', values='rating', aggfunc='mean').reset_index()\
.sort_values(by='rating', ascending=False)
coffee_rating.columns = ['district', 'avg_rating']

In [ ]:
coffee_rating

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
f_Choropleth(coffee_rating,['district', 'avg_rating'],'плотность распределения кофеен по районам')

##### 4.4 На какую стоимость чашки капучино стоит ориентироваться при открытии и почему.

In [ ]:
coffee_price = df.query('category == "кофейня"')\
.pivot_table(index='district_cont', values='middle_coffee_cup', aggfunc=['mean','count']).reset_index()
coffee_price.columns = ['district', 'mean_bill', 'count']
coffee_price = coffee_price.sort_values(by='mean_bill', ascending=False)
coffee_price

150 - 200 рублей оптимальная цена за кружку кофе.
Самый дорогой стаканчик кофе - на западе, а дешевый - в южной части города

In [ ]:
display_barplot(coffee_price, 'district', 'mean_bill',\
               'Средняя цена за чашку капучино по округам',\
               'Средняя цена за чашку капучино',\
               'Название округа',
               140, 200)

В зависимости от округа открытия заведения возможно следует задать цены немного ниже средних по округу для привлечения потока первых клиентов.
Даже при отстутствии конкуренции стоит учитывать, что, в центральном округе очень много кофеен, и необходимо будет приложить очень много усилий на привлечении первых клиентов. 

##### 4.5 Построить визуализации. Дать рекомендацию для открытия нового заведения.

В качестве наглядного примера был выбран ЦАО, так как ценник на кружку кофе в этом районе достаточно высок, что позволит создать конкурентноспособный ценник для привлечения новых клиентов  а соотношение кофеен к другому типа заведений не так высока как в остальных административных округах - примерно из 5 заведений - только 1 является кофейней (20,7%).

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423

m = Map(location=[moscow_lat, moscow_lng], zoom_start=11)
marker_cluster = MarkerCluster().add_to(m)
def create_marker(row):
    Marker([row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}"
    ).add_to(m)

df.query('category == "кофейня" and district == "Центральный административный округ"').apply(create_marker, axis=1)

m

### Вывод

Для работы был получен датафрейм с заведениях общественного питания в Москве.

Была проведена предобработка данных: 

- Отработаны пропуски, внесены необходимые изменения в данные.
- Изучены дубликаты в данных.
- Создан столбец street с названиями улиц из столбца с адресом.
- Создан столбец is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно (24/7)

Была построена визуализация по категориям заведений, которая показала, что наиболее популярные категории заведений:

- кафе – 2378 заведений
- ресторан – 2043 заведений
- кофейня – 1413 заведений

По количеству посадочных мест по медианным значениям топ возглавляют следующие 3 категории заведений:

- ресторан – 86 посадочных мест по медиане
- бар/паб – 82 посадочных места по медиане
- кофейня – 80 посадочных мест по медиане

По полученным данным, можно сделать вывод о том, что в данных категориях заведений люди любят проводить длительное время, чем и обосновывается количество посадочных мест.

Несетевых заведений в данных больше, а именно 5201 заведение, что составляет - 62%

Соотношение сетевых заведений к общему датафрейму:

- кофейня – 45% (350)
- ресторан – 20% (157)
- пиццерия – 19,5% (150)
- кафе – 11,5% (89)
- булочная – 3.2% (25)

Центральный административный округ сильно отличается от всех остальных районов распределением по категориям заведений – в этом округе значительно больше баров и ресторанов, а популярность кафе снижена.

Центральный и западный административные округа обладают самыми высокими ценами, а юго-восточный и южный административные округа самыми низкими.

##### Портрет потенциально успешного заведения по проанализированным данным

**Портрет** потенциально **успешного** заведения будет состоять:

**Тип заведения - кофейня**

**Время работы заведения - 24/7**

**Месторасположение - ЦАО** или ЗАО, если необходимо тратить меньше ресурсов на борьбу с огромной конкуренцией в Центральном округе

**Наличие посадочных мест**

**Средний чек - в рассматриваемых районах - 1000**, не стоит забывать о "сарафанном радио", как минимум на первое время - конкурентноспособная цена позволит набрать постоянных клиентов за достаточно короткий срок

Основной **конкурент - сетевые заведения** в выбранных районах, так что при выборе места заведения, стоит учитывать этот фактор

**Сетевое развитие - реально** в исполнении

### 5. Подготовка презентации

##### Задача:

- Подготовить презентацию исследования для инвесторов. 

##### Цель:

- Приложить ссылку на презентацию

https://disk.yandex.ru/i/XFtKfpGvTGRGNw